# Chapter 16. Logistic Regression

## 16.0 Introduction

## 16.1 Training a Binary Classifier

* Simple classifier model
* 사이킷런의 LogisticRegression 으로 로지스틱 회귀를 훈련한다.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data[:100, :]
target = iris.target[:100]

# Standardize features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# logistic regression object
logistic_regression = LogisticRegression(random_state=0)

# Train model
model = logistic_regression.fit(features_standardized, target)

/Users/statstics/anaconda3/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


* 로지스틱 회귀는 이진 분류기이다.
* 선형 모델로 로지스틱 함수(sigmoid function)에 포함된다.

$$P(y_i = 1 | X) = {{1}\over {1+e^{-(\beta_0 + \beta_1 x)}}}$$

* 로지스틱 함수로, 0과 1 사이 값으로 결과를 제한하여 확률로 해석되게 한다.
    * 0.5 를 넘으면 class 1이고, 아니면 class 0 으로 예측한다.

In [2]:
new_observation = [[.5,.5,.5,.5]]

model.predict(new_observation)

array([1])

In [3]:
# View predicted probabilities
model.predict_proba(new_observation)

array([[0.18944274, 0.81055726]])

## 16.2 Training a Multiclass Classifier

* 2 클래스 이상에서 분류기 모형으로 훈련한다.
* 로지스틱회귀로 1대 나머지를 활용하거나 multinomial 방법을 이용한다.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data
target = iris.target

# Standardize features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# one-vs-rest 로지스틱 회귀 객체를 만든다. multi_class="ovr" 이 핵심이다.
logistic_regression = LogisticRegression(random_state=0, multi_class="ovr")

model = logistic_regression.fit(features_standardized, target)

$$P(y_i = k | X) = {{e^{\beta_k x_i}} \over {\sum_{j=1}^K e^{\beta_j x_i}}}$$

* LogisticRegression
    * One-vs-Rest(binary) : ovr for OVR(default)
    * MLR by setting to multinomial

## 16.3 Reducing Variance Through Regularization
* Tune the Regularization strength hyperparameter C

In [1]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# iris
iris = datasets.load_iris()
features = iris.data
target = iris.target

# Standardized features :: 로지스틱 회귀할 때 표준화를 해준다.
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Create decision tree regression object
logistic_regression = LogisticRegressionCV(
    penalty='l2', Cs=10, random_state=0, n_jobs=-1)

# Train model
model = logistic_regression.fit(features_standardized, target)

* Regularization : 분산을 줄이기 위해 복잡한 모델에 패널티를 주는 방식이다.
    * Penaly term이 최소화하고자 하는 손실 함수에 더해진다.
    * 전형적으로 L1, L2 패널티를 사용한다.
* L1 패널티
$$\alpha \sum_{j=1}^p |\hat{\beta}_j|$$

 * $\hat{\beta}_j$ : 학습된 p개의 피처들의 j번째 파라미터
 * $\alpha$ : 정규화 강도를 표기한 하이퍼파라미터

* L2 패널티
$$\alpha \sum_{j=1}^p \hat{\beta}^2_j$$

* $\alpha$ 큰 값은 더 큰 파라미터 값에 대해 피널티를 증가시킨다.
* 사이킷런에서는 C를 $\alpha$ 대신 사용하며, 정규화 강도의 역순이 된다.  
* $C = {1\over \alpha}$

## 16.4 Training a Classifier on Very Large Data

* simple classifier model on very large dataset
    * logistic regression using the stochastic average gradient(SAG) solver(사이킷런의 함수, 파라미터)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data
target = iris.target

scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 로지스틱 회귀 객체 생성
logistic_regression = LogisticRegression(random_state=0, solver="sag")

# 모델 훈련
model = logistic_regression.fit(features_standardized, target)

* 사이킷런의 로지스틱 회귀 함수가 로지스틱 회귀를 훈련시키기 위한 많은 테크닉을 제공한다.
    * 이를 solver 라고 한다.
    * 자동적으로 최상의 solver를 선택했고, solver를 위해 우리가 할 필요가 없다. 그러나 한 가지는 알고 가자.
* Stochastic average gradient descent
    * 데이터가 매우 클 때, 특히 다른 solver보다 훨씬 더 빠르게 모델을 훈련한다.
    * 그러나 피처 스케일링에 매우 민감하므로, 표준화하는 게 특히 중요하다.
    * 이 알고리즘은 solver = 'sag' 로 활용할 수 있다.

## 16.5 Handling Imbalanced Classes
* Train a simple classifier model : 사이킷런의 LogisticRegression 활용

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data
target = iris.target

features = features[40:, :]
target = target[40:]

# Create target vector if class 0, 나머지는 모두 1로 변경한다.
target = np.where((target==0),0, 1)

# 피처를 표준화한다.
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Create Decision Tree regresssion object
logistic_regression = LogisticRegression(random_state=0, class_weight="balanced")

# Train model
model = logistic_regression.fit(features_standardized, target)

In [9]:
model

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

* 이 외에도 다양한 학습 알고리즘이 사이킷런에 있다.
    * LogisticRegression : 내장된 방식으로 불균형 클래스를 처리한다.
    * 매우 불균형 한 경우, 활용할 수 없다면, class_weight 파라미터를 활용한다.
        * 이를 활용하면, 균형된 argument가 자동적으로 weigh class를 역으로 비율에 비례한다.

$$w_j = {{n} \over {kn_j}}$$